In [1]:
!pwd

/content


In [2]:
!ls

drive  sample_data


In [3]:
cd drive

/content/drive


In [4]:
!ls

'My Drive'


In [5]:
cd 'My Drive'

/content/drive/My Drive


In [6]:
!ls

'Colab Notebooks'   deep-learning-from-scratch-master   IMG_0001.pdf


In [0]:
#cd 'Colab Notebooks'

In [0]:
#!pwd

In [0]:
#!ls

In [10]:
cd deep-learning-from-scratch-master

/content/drive/My Drive/deep-learning-from-scratch-master


In [11]:
!ls

 ch01   ch05   common	   't10k-images-idx3-ubyte.gz のコピー'
 ch02   ch06   dataset	   't10k-labels-idx1-ubyte.gz のコピー'
 ch03   ch07   LICENSE.md  'train-images-idx3-ubyte.gz のコピー'
 ch04   ch08   README.md   'train-labels-idx1-ubyte.gz のコピー'


In [12]:
cd ch07

/content/drive/My Drive/deep-learning-from-scratch-master/ch07


In [0]:
import numpy as np
x = np.random.rand(10, 1, 28, 28) #ランダムにデータを生成

In [16]:
x.shape

(10, 1, 28, 28)

In [17]:
x[0].shape

(1, 28, 28)

In [18]:
x[1].shape

(1, 28, 28)

In [19]:
x[0, 0] #もしくはx[0][0]

array([[3.03543567e-01, 8.84049002e-01, 5.09068215e-01, 6.83350382e-01,
        3.08975693e-01, 2.73880884e-01, 5.72637017e-01, 9.10417921e-02,
        3.74435216e-01, 7.38633368e-01, 3.87789805e-01, 4.62858319e-01,
        3.25793256e-01, 9.56634933e-01, 9.82041888e-01, 4.83580137e-01,
        6.68139309e-01, 7.98135649e-01, 8.48772817e-01, 7.28607338e-01,
        7.78625753e-01, 5.27977144e-02, 4.70336675e-01, 5.84103692e-01,
        6.77578416e-02, 6.71867984e-02, 9.31383314e-01, 1.38196748e-01],
       [9.75222352e-01, 4.18005482e-01, 9.54647011e-01, 1.80909104e-02,
        8.90787682e-01, 5.68482602e-01, 8.37460659e-01, 1.02394226e-01,
        3.51804430e-01, 1.33912655e-02, 6.33922729e-02, 6.51427107e-01,
        6.42169394e-01, 4.68757279e-01, 1.56006213e-01, 6.93492304e-01,
        2.14453645e-01, 8.95217171e-01, 3.18554816e-01, 9.90404740e-01,
        1.71447254e-01, 6.27921774e-01, 2.23510756e-01, 5.42085723e-01,
        2.32176081e-01, 5.67674988e-02, 9.05360995e-01, 3.79812

In [24]:
#im2col(input_data, filter_h, filter_w, stride=1, pad=0)

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) #10個のデータ
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [0]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
  
  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
    
    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T #フィルターの展開
    out = np.dot(col, col_W) + self.b
    
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    
    return out

In [0]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
    
  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)
    
    #展開(1)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)
    
    #最大値(2)
    out = np.max(col, axis=1)
    #整形(3)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    
    return out

In [0]:
class SimpleConvNet:
  def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
    pool_output_size = int(filter_num * (conv_output_seize/2) * (conv_output_size/2))
    
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)
    
    self.layers = OrederedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    
    def predict(self, x):
      for layer in self.layers.values():
        x = layer.forward(x)
      return x
    
    def loss(self, x, t):
      y = self.predict(x)
      return self.lastLayer.forward(y, t)
    
    def gradient(self, x, t):
      #forward
      self.loss(x, t)
      
      #backward
      dout = 1
      dout = self.lastLayer.backward(dout)
      
      layers = list(self.layers.values())
      layers.reverse()
      for layer in layers:
        dout = layer.backward(dout)
        
      #設定
      grads = {}
      grads['W1'] = self.layers['Conv1'].dW
      grads['b1'] = self.layers['Cinv1'].db
      grads['W2'] = self.layers['Affine1'].dW
      grads['b2'] = self.layers['Affine1'].db
      grads['W3'] = self.layers['Affine2'].dW
      grads['b3'] = self.layers['Affine2'].db
      
      return grads

In [0]:
#cd /content